[View in Colaboratory](https://colab.research.google.com/github/LeonidSLA/keras_captcha/blob/master/main.ipynb)

In [4]:
!pip install -q xlrd
!pip install captcha

In [5]:
ls

datalab/  keras_captcha/


In [2]:
!git clone https://github.com/LeonidSLA/keras_captcha.git

Cloning into 'keras_captcha'...
remote: Counting objects: 2571, done.
remote: Compressing objects: 100% (2540/2540), done.
remote: Total 2571 (delta 33), reused 2561 (delta 23), pack-reused 0
Receiving objects: 100% (2571/2571), 72.39 MiB | 37.17 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [132]:
!git clean -d -f
!git pull origin master

fatal: Not a git repository (or any of the parent directories): .git
fatal: Not a git repository (or any of the parent directories): .git


In [0]:
import os
os.chdir('keras_captcha')

In [131]:
ls

datalab/  keras_captcha/


In [0]:
rm -r keras_captcha

In [75]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18396 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Preparing to unpack .../04-gnupg_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking gnupg (2.1.15-1ubuntu8.1) over (2.1.15-1ubuntu8) ...
Preparing to unpack .../05-gnupg-agent_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking gnupg-agent (2.1.15-1ubuntu8

Setting up google-drive-ocamlfuse (0.6.21-0ubuntu2) ...


KeyboardInterrupt: ignored

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
ls

datalab/


In [0]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

import captcha_params

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.optimizers import adamax
from keras.callbacks import Callback
from load_data import load_data
import keras
import get_train_set
#import h5py
#from keras.models import model_from_json

In [135]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
batch_size = 10
nb_epoch = 100
MAX_CAPTCHA = captcha_params.get_captcha_size()
CHAR_SET_LEN = captcha_params.get_char_set_len()

# input image dimensions
img_rows, img_cols = captcha_params.get_height(), captcha_params.get_width()

In [0]:
#Generate additional training set
for i in range(5000):     
    text, image = get_train_set.gen_captcha_text_and_image(i)


In [0]:
#original data
from load_data import load_data
(X_train, y_train,y_train_text), (X_test, y_test,y_test_text) = load_data(tol_num = 1000,train_num = 700,folder='labeled_captchas')

In [0]:
# Generated data
(X_train_gen, y_train_gen,y_train_text_gen), (X_test_gen, y_test_gen,y_test_text_gen) = load_data(tol_num = 2000,train_num = 2000,folder='generated')

In [0]:
np.save('X_train_gen.npy',X_train_gen)
np.save('y_train_gen.npy',y_train_gen)
#y_train_gen.save('y_train_gen.dat')

In [0]:
X_train_gen = np.load('X_train_gen.npy')
y_train_gen = np.load('y_train_gen.npy')
#y_train_gen = np.fromfile('y_train_gen.dat')

In [0]:
# Add generated data to train set

X_train=np.append(X_train, X_train_gen,axis=0)
y_train=np.append(y_train, y_train_gen,axis=0)
#y_train_text=np.append(y_train_text, y_train_text_gen)

In [0]:
y_train=y_train.reshape(y_train.shape[0],5,36)
y_test=y_test.reshape(y_test.shape[0],5,36)

In [0]:
input_shape = (img_rows, img_cols, 3)

In [141]:
X_train.shape

(700, 150, 330, 3)

In [0]:
train_lst=[y_train[:,0,] ,
           y_train[:,1,] , 
           y_train[:,2,] ,
           y_train[:,3,] ,
           y_train[:,4,]  
          ]

test_lst=[y_test[:,0,] ,
           y_test[:,1,] , 
           y_test[:,2,] ,
           y_test[:,3,] ,
           y_test[:,4,]  
          ]

In [0]:
#model = Sequential()

# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)
batch_size =70
nb_filters1=48
nb_filters=24
nb_filters1=16

In [0]:
from keras.layers import Input
from keras.models import Model

batch_size = 100
nb_classes = 36
nb_epoch = 100


model_input=Input(shape=input_shape)

x = MaxPooling2D(pool_size=(2, 2))(model_input)


x = Conv2D(24, kernel_size)(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)



x = Conv2D(12, kernel_size)(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)



x = Conv2D(12, kernel_size)(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)


conv_out = Flatten()(x)

#y = Dense(256, activation="relu")(y)

conv_out = Dense(100)(conv_out)
conv_out = Activation('relu')(conv_out)
conv_out = Dropout(0.25)(conv_out)

x1 = Dense(nb_classes, activation='softmax')(conv_out)
x2 = Dense(nb_classes, activation='softmax')(conv_out)
x3 = Dense(nb_classes, activation='softmax')(conv_out)
x4 = Dense(nb_classes, activation='softmax')(conv_out)
x5 = Dense(nb_classes, activation='softmax')(conv_out)

lst = [x1, x2, x3, x4, x5]



model = Model(inputs=model_input, outputs=lst)

In [0]:
from keras.losses import categorical_crossentropy
def m_losses(y_true,y_pred):
    loss =categorical_crossentropy(y_true[0], y_pred[0])+categorical_crossentropy(y_true[1], y_pred[1])+categorical_crossentropy(y_true[2], y_pred[2])+categorical_crossentropy(y_true[3], y_pred[3])+categorical_crossentropy(y_true[4], y_pred[4])
    
    return loss

In [0]:
def inception(use_imagenet=True):
    # load pre-trained model graph, don't add final layer
    model = keras.applications.InceptionV3(include_top=False, input_shape=input_shape,
                                          weights='imagenet' if use_imagenet else None)
    # add global pooling just like in InceptionV3
    new_output = keras.layers.GlobalAveragePooling2D()(model.output)
    # add new dense layer for our labels
    new_output = keras.layers.Dense(MAX_CAPTCHA*CHAR_SET_LEN, activation='softmax')(new_output)
    model = keras.engine.training.Model(model.inputs, new_output)
    return model

In [27]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 150, 330, 3)  0                                            
__________________________________________________________________________________________________
max_pooling2d_24 (MaxPooling2D) (None, 75, 165, 3)   0           input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 73, 163, 24)  672         max_pooling2d_24[0][0]           
__________________________________________________________________________________________________
activation_24 (Activation)      (None, 73, 163, 24)  0           conv2d_19[0][0]                  
__________________________________________________________________________________________________
max_poolin

In [0]:
model = inception()

87916544/87910968 [==============================] - 6s 0us/step


In [0]:
# fix deep layers (fine-tuning only last 50)
for layer in model.layers[:-50]:
    layer.trainable = False

In [0]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer=keras.optimizers.adamax(lr=1e-2),  # we can take big lr here because we fixed first layers
    metrics=['accuracy']  # report accuracy during training
)

In [0]:
model.add(Conv2D(kernel_size=kernel_size, filters=nb_filters1, padding='same' ,input_shape=input_shape ))
model.add(LeakyReLU(0.1))

model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Conv2D( kernel_size=kernel_size, filters=nb_filters2))
model.add(LeakyReLU(0.1))

model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))


# Fully connected layer
model.add(Flatten())

model.add(Dense(512))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.25))

model.add(Dense(MAX_CAPTCHA*CHAR_SET_LEN))

model.add(Activation('softmax'))

In [78]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 150, 330, 3)  0                                            
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 150, 330, 32) 896         input_9[0][0]                    
__________________________________________________________________________________________________
activation_22 (Activation)      (None, 150, 330, 32) 0           conv2d_17[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_16 (MaxPooling2D) (None, 75, 165, 32)  0           activation_22[0][0]              
__________________________________________________________________________________________________
dropout_21

In [0]:
batch_size =10

nb_epoch = 100

model.compile(loss=m_losses,
              optimizer=adamax(lr=0.0001),
              metrics=['accuracy'])


In [30]:
model.fit(X_train, train_lst, batch_size=batch_size, epochs=nb_epoch,
            verbose=1, validation_data=(X_test,test_lst))

Train on 700 samples, validate on 300 samples
Epoch 1/100
700/700 [==============================] - 6s 8ms/step - loss: 383.8260 - dense_38_loss: 75.1233 - dense_39_loss: 78.9787 - dense_40_loss: 78.9787 - dense_41_loss: 74.8305 - dense_42_loss: 75.9149 - dense_38_acc: 0.0486 - dense_39_acc: 0.0143 - dense_40_acc: 0.0100 - dense_41_acc: 0.0671 - dense_42_acc: 0.0429 - val_loss: 378.6880 - val_dense_38_loss: 72.5314 - val_dense_39_loss: 78.9787 - val_dense_40_loss: 78.9787 - val_dense_41_loss: 72.5314 - val_dense_42_loss: 75.6678 - val_dense_38_acc: 0.0700 - val_dense_39_acc: 0.0000e+00 - val_dense_40_acc: 0.0000e+00 - val_dense_41_acc: 0.0767 - val_dense_42_acc: 0.0567
Epoch 2/100
700/700 [==============================] - 3s 4ms/step - loss: 389.2146 - dense_38_loss: 75.7291 - dense_39_loss: 80.1300 - dense_40_loss: 79.8997 - dense_41_loss: 75.9358 - dense_42_loss: 77.5200 - dense_38_acc: 0.0571 - dense_39_acc: 0.0043 - dense_40_acc: 0.0114 - dense_41_acc: 0.0586 - dense_42_acc: 0.02

700/700 [==============================] - 3s 4ms/step - loss: 381.0426 - dense_38_loss: 74.1656 - dense_39_loss: 78.9828 - dense_40_loss: 78.6815 - dense_41_loss: 74.2357 - dense_42_loss: 74.9770 - dense_38_acc: 0.0614 - dense_39_acc: 0.0114 - dense_40_acc: 0.0157 - dense_41_acc: 0.0671 - dense_42_acc: 0.0443 - val_loss: 378.1865 - val_dense_38_loss: 72.5314 - val_dense_39_loss: 78.5317 - val_dense_40_loss: 78.9787 - val_dense_41_loss: 72.5314 - val_dense_42_loss: 75.6133 - val_dense_38_acc: 0.0700 - val_dense_39_acc: 0.0000e+00 - val_dense_40_acc: 0.0000e+00 - val_dense_41_acc: 0.0767 - val_dense_42_acc: 0.0567
Epoch 4/100
700/700 [==============================] - 3s 4ms/step - loss: 386.5417 - dense_38_loss: 74.5550 - dense_39_loss: 79.4736 - dense_40_loss: 79.9019 - dense_41_loss: 75.2342 - dense_42_loss: 77.3771 - dense_38_acc: 0.0629 - dense_39_acc: 0.0143 - dense_40_acc: 0.0043 - dense_41_acc: 0.0686 - dense_42_acc: 0.0357 - val_loss: 377.7776 - val_dense_38_loss: 72.5314 - val

700/700 [==============================] - 3s 4ms/step - loss: 388.3159 - dense_38_loss: 76.5119 - dense_39_loss: 79.2673 - dense_40_loss: 79.7719 - dense_41_loss: 75.9853 - dense_42_loss: 76.7795 - dense_38_acc: 0.0557 - dense_39_acc: 0.0143 - dense_40_acc: 0.0057 - dense_41_acc: 0.0600 - dense_42_acc: 0.0457 - val_loss: 376.7936 - val_dense_38_loss: 72.5314 - val_dense_39_loss: 77.3669 - val_dense_40_loss: 78.6088 - val_dense_41_loss: 72.5314 - val_dense_42_loss: 75.7550 - val_dense_38_acc: 0.0700 - val_dense_39_acc: 0.0167 - val_dense_40_acc: 0.0000e+00 - val_dense_41_acc: 0.0767 - val_dense_42_acc: 0.0600
Epoch 7/100
700/700 [==============================] - 3s 4ms/step - loss: 383.0461 - dense_38_loss: 76.4728 - dense_39_loss: 78.7484 - dense_40_loss: 78.6245 - dense_41_loss: 74.1867 - dense_42_loss: 75.0137 - dense_38_acc: 0.0529 - dense_39_acc: 0.0157 - dense_40_acc: 0.0129 - dense_41_acc: 0.0657 - dense_42_acc: 0.0614 - val_loss: 376.0482 - val_dense_38_loss: 72.5314 - val_den

700/700 [==============================] - 3s 4ms/step - loss: 383.5881 - dense_38_loss: 75.0536 - dense_39_loss: 78.5598 - dense_40_loss: 78.4646 - dense_41_loss: 74.8340 - dense_42_loss: 76.6761 - dense_38_acc: 0.0586 - dense_39_acc: 0.0157 - dense_40_acc: 0.0129 - dense_41_acc: 0.0571 - dense_42_acc: 0.0329 - val_loss: 376.2492 - val_dense_38_loss: 72.5314 - val_dense_39_loss: 77.1978 - val_dense_40_loss: 78.4201 - val_dense_41_loss: 72.5314 - val_dense_42_loss: 75.5685 - val_dense_38_acc: 0.0700 - val_dense_39_acc: 0.0233 - val_dense_40_acc: 0.0067 - val_dense_41_acc: 0.0767 - val_dense_42_acc: 0.0600
Epoch 9/100
700/700 [==============================] - 3s 4ms/step - loss: 385.4975 - dense_38_loss: 75.3209 - dense_39_loss: 78.9338 - dense_40_loss: 79.2089 - dense_41_loss: 74.8712 - dense_42_loss: 77.1627 - dense_38_acc: 0.0643 - dense_39_acc: 0.0186 - dense_40_acc: 0.0143 - dense_41_acc: 0.0614 - dense_42_acc: 0.0371 - val_loss: 374.8279 - val_dense_38_loss: 72.5314 - val_dense_3

700/700 [==============================] - 3s 4ms/step - loss: 386.0419 - dense_38_loss: 74.9055 - dense_39_loss: 79.0003 - dense_40_loss: 79.1400 - dense_41_loss: 76.2286 - dense_42_loss: 76.7675 - dense_38_acc: 0.0714 - dense_39_acc: 0.0214 - dense_40_acc: 0.0100 - dense_41_acc: 0.0629 - dense_42_acc: 0.0457 - val_loss: 374.3701 - val_dense_38_loss: 72.5314 - val_dense_39_loss: 76.4907 - val_dense_40_loss: 77.3912 - val_dense_41_loss: 72.5314 - val_dense_42_loss: 75.4253 - val_dense_38_acc: 0.0700 - val_dense_39_acc: 0.0267 - val_dense_40_acc: 0.0167 - val_dense_41_acc: 0.0767 - val_dense_42_acc: 0.0533
Epoch 12/100
700/700 [==============================] - 3s 4ms/step - loss: 382.1247 - dense_38_loss: 74.4993 - dense_39_loss: 78.9787 - dense_40_loss: 78.4537 - dense_41_loss: 74.0297 - dense_42_loss: 76.1633 - dense_38_acc: 0.0657 - dense_39_acc: 0.0143 - dense_40_acc: 0.0129 - dense_41_acc: 0.0671 - dense_42_acc: 0.0514 - val_loss: 373.2940 - val_dense_38_loss: 72.5314 - val_dense_

...

700/700 [==============================] - 3s 4ms/step - loss: 378.2131 - dense_38_loss: 73.8510 - dense_39_loss: 76.6761 - dense_40_loss: 77.2232 - dense_41_loss: 75.2838 - dense_42_loss: 75.1791 - dense_38_acc: 0.0700 - dense_39_acc: 0.0571 - dense_40_acc: 0.0400 - dense_41_acc: 0.0657 - dense_42_acc: 0.0586 - val_loss: 369.6417 - val_dense_38_loss: 72.5314 - val_dense_39_loss: 75.2178 - val_dense_40_loss: 73.6060 - val_dense_41_loss: 72.5314 - val_dense_42_loss: 75.7550 - val_dense_38_acc: 0.0700 - val_dense_39_acc: 0.0500 - val_dense_40_acc: 0.0600 - val_dense_41_acc: 0.0767 - val_dense_42_acc: 0.0600
Epoch 82/100
700/700 [==============================] - 3s 4ms/step - loss: 376.3286 - dense_38_loss: 75.5755 - dense_39_loss: 75.7550 - dense_40_loss: 75.4512 - dense_41_loss: 74.8858 - dense_42_loss: 74.6610 - dense_38_acc: 0.0686 - dense_39_acc: 0.0500 - dense_40_acc: 0.0457 - dense_41_acc: 0.0629 - dense_42_acc: 0.0571 - val_loss: 369.6417 - val_dense_38_loss: 72.5314 - val_dense_

700/700 [==============================] - 3s 4ms/step - loss: 373.7129 - dense_38_loss: 73.7480 - dense_39_loss: 75.8781 - dense_40_loss: 76.8960 - dense_41_loss: 73.5094 - dense_42_loss: 73.6814 - dense_38_acc: 0.0686 - dense_39_acc: 0.0529 - dense_40_acc: 0.0429 - dense_41_acc: 0.0657 - dense_42_acc: 0.0729 - val_loss: 369.6417 - val_dense_38_loss: 72.5314 - val_dense_39_loss: 75.2178 - val_dense_40_loss: 73.6060 - val_dense_41_loss: 72.5314 - val_dense_42_loss: 75.7550 - val_dense_38_acc: 0.0700 - val_dense_39_acc: 0.0500 - val_dense_40_acc: 0.0600 - val_dense_41_acc: 0.0767 - val_dense_42_acc: 0.0600
Epoch 84/100
700/700 [==============================] - 3s 4ms/step - loss: 378.4055 - dense_38_loss: 73.9734 - dense_39_loss: 76.2156 - dense_40_loss: 76.3899 - dense_41_loss: 76.9595 - dense_42_loss: 74.8672 - dense_38_acc: 0.0700 - dense_39_acc: 0.0543 - dense_40_acc: 0.0529 - dense_41_acc: 0.0586 - dense_42_acc: 0.0671 - val_loss: 369.6417 - val_dense_38_loss: 72.5314 - val_dense_

700/700 [==============================] - 3s 4ms/step - loss: 375.9586 - dense_38_loss: 74.6551 - dense_39_loss: 76.0255 - dense_40_loss: 76.4473 - dense_41_loss: 73.7907 - dense_42_loss: 75.0401 - dense_38_acc: 0.0657 - dense_39_acc: 0.0500 - dense_40_acc: 0.0471 - dense_41_acc: 0.0700 - dense_42_acc: 0.0529 - val_loss: 369.6417 - val_dense_38_loss: 72.5314 - val_dense_39_loss: 75.2178 - val_dense_40_loss: 73.6060 - val_dense_41_loss: 72.5314 - val_dense_42_loss: 75.7550 - val_dense_38_acc: 0.0700 - val_dense_39_acc: 0.0500 - val_dense_40_acc: 0.0600 - val_dense_41_acc: 0.0767 - val_dense_42_acc: 0.0600
Epoch 87/100
700/700 [==============================] - 3s 4ms/step - loss: 382.2359 - dense_38_loss: 76.6332 - dense_39_loss: 76.4458 - dense_40_loss: 78.0577 - dense_41_loss: 74.8978 - dense_42_loss: 76.2013 - dense_38_acc: 0.0586 - dense_39_acc: 0.0529 - dense_40_acc: 0.0400 - dense_41_acc: 0.0629 - dense_42_acc: 0.0500 - val_loss: 369.6417 - val_dense_38_loss: 72.5314 - val_dense_

700/700 [==============================] - 3s 4ms/step - loss: 375.7491 - dense_38_loss: 74.0668 - dense_39_loss: 76.9066 - dense_40_loss: 75.5248 - dense_41_loss: 74.6038 - dense_42_loss: 74.6472 - dense_38_acc: 0.0729 - dense_39_acc: 0.0529 - dense_40_acc: 0.0529 - dense_41_acc: 0.0714 - dense_42_acc: 0.0543 - val_loss: 369.6417 - val_dense_38_loss: 72.5314 - val_dense_39_loss: 75.2178 - val_dense_40_loss: 73.6060 - val_dense_41_loss: 72.5314 - val_dense_42_loss: 75.7550 - val_dense_38_acc: 0.0700 - val_dense_39_acc: 0.0500 - val_dense_40_acc: 0.0600 - val_dense_41_acc: 0.0767 - val_dense_42_acc: 0.0600
Epoch 89/100
700/700 [==============================] - 3s 4ms/step - loss: 377.5082 - dense_38_loss: 75.0021 - dense_39_loss: 76.6761 - dense_40_loss: 75.3713 - dense_41_loss: 73.4525 - dense_42_loss: 77.0064 - dense_38_acc: 0.0686 - dense_39_acc: 0.0514 - dense_40_acc: 0.0471 - dense_41_acc: 0.0671 - dense_42_acc: 0.0529 - val_loss: 369.6417 - val_dense_38_loss: 72.5314 - val_dense_

KeyboardInterrupt: ignored

In [0]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,
            verbose=1, validation_data=(X_test,y_test))

Train on 5500 samples, validate on 500 samples
Epoch 1/100
5500/5500 [==============================] - 43s 8ms/step - loss: 25.2689 - acc: 0.0204 - val_loss: 22.1700 - val_acc: 0.0280
Epoch 2/100
5500/5500 [==============================] - 34s 6ms/step - loss: 21.8578 - acc: 0.0862 - val_loss: 22.6610 - val_acc: 0.0640
Epoch 3/100
5500/5500 [==============================] - 35s 6ms/step - loss: 18.8426 - acc: 0.1680 - val_loss: 22.4841 - val_acc: 0.0480
Epoch 4/100
5500/5500 [==============================] - 35s 6ms/step - loss: 16.1464 - acc: 0.2247 - val_loss: 24.6651 - val_acc: 0.0100
Epoch 5/100
3000/5500 [===============>..............] - ETA: 14s - loss: 14.2210 - acc: 0.2443

5500/5500 [==============================] - 35s 6ms/step - loss: 14.2773 - acc: 0.2415 - val_loss: 26.7780 - val_acc: 0.0260
Epoch 6/100
5500/5500 [==============================] - 35s 6ms/step - loss: 13.1220 - acc: 0.2347 - val_loss: 29.0632 - val_acc: 0.0480
Epoch 7/100
5500/5500 [==============================] - 35s 6ms/step - loss: 12.4929 - acc: 0.2282 - val_loss: 34.5680 - val_acc: 0.0220
Epoch 8/100
5500/5500 [==============================] - 35s 6ms/step - loss: 12.0773 - acc: 0.2275 - val_loss: 34.1477 - val_acc: 0.0060
Epoch 9/100
5500/5500 [==============================] - 35s 6ms/step - loss: 11.7684 - acc: 0.2129 - val_loss: 35.1606 - val_acc: 0.0280
Epoch 10/100
 100/5500 [..............................] - ETA: 30s - loss: 11.4504 - acc: 0.1700

5500/5500 [==============================] - 35s 6ms/step - loss: 11.5018 - acc: 0.2198 - val_loss: 32.9730 - val_acc: 0.0020
Epoch 11/100
5500/5500 [==============================] - 35s 6ms/step - loss: 11.2834 - acc: 0.2125 - val_loss: 35.7876 - val_acc: 0.0000e+00
Epoch 12/100
5500/5500 [==============================] - 35s 6ms/step - loss: 11.1757 - acc: 0.2089 - val_loss: 32.1775 - val_acc: 0.0160
Epoch 13/100
5500/5500 [==============================] - 34s 6ms/step - loss: 11.0493 - acc: 0.2127 - val_loss: 32.8633 - val_acc: 0.0060
Epoch 14/100
5200/5500 [===========================>..] - ETA: 1s - loss: 10.9204 - acc: 0.2052

5500/5500 [==============================] - 35s 6ms/step - loss: 10.9177 - acc: 0.2044 - val_loss: 30.0162 - val_acc: 0.0240
Epoch 15/100
5500/5500 [==============================] - 35s 6ms/step - loss: 10.7762 - acc: 0.2058 - val_loss: 34.8518 - val_acc: 0.0060
Epoch 16/100
5500/5500 [==============================] - 35s 6ms/step - loss: 10.7078 - acc: 0.2122 - val_loss: 35.9082 - val_acc: 0.0060
Epoch 17/100
5500/5500 [==============================] - 35s 6ms/step - loss: 10.6309 - acc: 0.2084 - val_loss: 34.8384 - val_acc: 0.0000e+00
Epoch 18/100
5500/5500 [==============================] - 35s 6ms/step - loss: 10.5851 - acc: 0.2060 - val_loss: 33.1622 - val_acc: 0.0080
Epoch 19/100
 300/5500 [>.............................] - ETA: 29s - loss: 10.3291 - acc: 0.2167

5500/5500 [==============================] - 35s 6ms/step - loss: 10.4627 - acc: 0.2102 - val_loss: 35.1813 - val_acc: 0.0100
Epoch 20/100
5500/5500 [==============================] - 35s 6ms/step - loss: 10.4272 - acc: 0.2087 - val_loss: 34.0734 - val_acc: 0.0140
Epoch 21/100
5500/5500 [==============================] - 35s 6ms/step - loss: 10.3665 - acc: 0.2067 - val_loss: 36.9413 - val_acc: 0.0020
Epoch 22/100
5500/5500 [==============================] - 35s 6ms/step - loss: 10.3583 - acc: 0.2080 - val_loss: 33.4993 - val_acc: 0.0160
Epoch 23/100
5400/5500 [============================>.] - ETA: 0s - loss: 10.2728 - acc: 0.1957

5500/5500 [==============================] - 35s 6ms/step - loss: 10.2691 - acc: 0.1944 - val_loss: 34.0805 - val_acc: 0.0140
Epoch 24/100
5500/5500 [==============================] - 34s 6ms/step - loss: 10.2188 - acc: 0.2024 - val_loss: 32.1892 - val_acc: 0.0380
Epoch 25/100
5500/5500 [==============================] - 34s 6ms/step - loss: 10.1730 - acc: 0.1996 - val_loss: 35.3467 - val_acc: 0.0120
Epoch 26/100
5500/5500 [==============================] - 35s 6ms/step - loss: 10.1747 - acc: 0.2025 - val_loss: 34.2580 - val_acc: 0.0040
Epoch 27/100
3400/5500 [=================>............] - ETA: 12s - loss: 10.1271 - acc: 0.1924

5500/5500 [==============================] - 35s 6ms/step - loss: 10.1311 - acc: 0.1956 - val_loss: 32.2648 - val_acc: 0.0220
Epoch 28/100
5500/5500 [==============================] - 35s 6ms/step - loss: 10.0653 - acc: 0.2040 - val_loss: 35.7962 - val_acc: 0.0140
Epoch 29/100
5500/5500 [==============================] - 35s 6ms/step - loss: 10.0472 - acc: 0.2044 - val_loss: 33.2968 - val_acc: 0.0060
Epoch 30/100
5500/5500 [==============================] - 35s 6ms/step - loss: 9.9931 - acc: 0.1924 - val_loss: 32.2726 - val_acc: 0.0160
Epoch 31/100
5500/5500 [==============================] - 35s 6ms/step - loss: 9.9273 - acc: 0.2040 - val_loss: 31.8521 - val_acc: 0.0120
Epoch 32/100
 100/5500 [..............................] - ETA: 30s - loss: 9.7580 - acc: 0.1400

5500/5500 [==============================] - 35s 6ms/step - loss: 9.9199 - acc: 0.2064 - val_loss: 31.7656 - val_acc: 0.0220
Epoch 33/100
5500/5500 [==============================] - 35s 6ms/step - loss: 9.9248 - acc: 0.1969 - val_loss: 32.0583 - val_acc: 0.0120
Epoch 34/100
5500/5500 [==============================] - 35s 6ms/step - loss: 9.8520 - acc: 0.2009 - val_loss: 32.0471 - val_acc: 0.0100
Epoch 35/100
3800/5500 [===================>..........] - ETA: 9s - loss: 9.8130 - acc: 0.2084 

KeyboardInterrupt: ignored

In [0]:
score = model.evaluate(X_test, y_test, verbose=0)
predict = model.predict(X_test,batch_size = batch_size,verbose = 0)

In [0]:
score

[11.118076133728028, 0.28]

In [0]:
predict[1]

array([1.5284390e-06, 2.3697584e-04, 1.8197912e-04, 5.8962614e-05,
       1.1963204e-03, 2.4681420e-03, 1.1096110e-03, 3.6078872e-04,
       3.8474204e-04, 1.6395837e-03, 1.4101419e-01, 7.6950423e-04,
       1.0255806e-03, 4.3321475e-03, 4.5343876e-04, 4.7576907e-03,
       1.6470024e-06, 2.5950649e-06, 2.2418210e-06, 2.7139024e-06,
       1.7650574e-06, 9.8350165e-06, 3.3705430e-06, 1.1283876e-06,
       1.3486898e-04, 4.4983476e-06, 2.9816508e-06, 4.5750594e-06,
       2.2231916e-06, 1.4044243e-06, 2.2699733e-06, 3.2218152e-06,
       3.5158307e-06, 1.4041725e-06, 2.2417439e-06, 5.8015248e-06,
       1.6450004e-03, 1.5243220e-03, 1.3717784e-03, 9.1513270e-04,
       1.3421621e-02, 3.8186298e-04, 1.7035551e-02, 4.1572613e-04,
       4.5722527e-03, 5.5355928e-04, 5.2857250e-03, 6.1748084e-04,
       1.0539745e-03, 1.4671421e-04, 2.7498386e-03, 4.4137653e-04,
       3.6473600e-06, 2.7770843e-06, 3.4474535e-06, 4.7992144e-06,
       3.3268238e-06, 2.2711015e-06, 6.2980062e-06, 2.9012922e